In [ ]:
#UPDATED 03/02/2025
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import io
import re

# Function to extract only numbers from mixed data
def extract_numbers(value):
    if isinstance(value, (int, float)):  # Directly return numbers if already numeric
        return int(value) if not pd.isna(value) else None
    elif isinstance(value, str):  # Extract numbers from strings
        numbers = re.findall(r'\d+', value)
        return int(numbers[0]) if numbers else None
    return None

# Function to process the uploaded file
def process_file(uploaded_file):
    try:
        # Extract the uploaded file content
        uploaded_filename = list(uploaded_file.keys())[0]  # Get the filename
        file_content = uploaded_file[uploaded_filename]['content']

        # Convert the uploaded file content into a bytes buffer
        file_stream = io.BytesIO(file_content)

        # Load the Excel file and get the first sheet name dynamically
        xls = pd.ExcelFile(file_stream)
        first_sheet_name = xls.sheet_names[0]  # Get the first sheet name
        print(f"\n📄 Processing file: {uploaded_filename}")
        print(f"📑 Using first sheet: {first_sheet_name}")

        # Read only the first column from the first sheet
        bk_stats_scanned = pd.read_excel(xls, sheet_name=first_sheet_name, header=None, usecols=[0])

        # Check if the sheet is empty
        if bk_stats_scanned.empty:
            print(f"\n⚠ The first sheet '{first_sheet_name}' in '{uploaded_filename}' is empty.")
            return

        # Extract numbers from the first column and remove NaN values
        bk_stats_scanned = bk_stats_scanned[0].map(extract_numbers)

        # Remove None values (which result from empty or non-numeric rows)
        bk_stats_scanned = bk_stats_scanned.dropna().astype(int).tolist()

        # Check if the list is still empty after conversion
        if not bk_stats_scanned:
            print(f"\n⚠ No valid numeric data in the first sheet '{first_sheet_name}' of '{uploaded_filename}'.")
            return

        # Sort the numbers
        bk_stats_scanned = sorted(bk_stats_scanned)

        # Get starting and ending numbers
        start_number = bk_stats_scanned[0]
        end_number = bk_stats_scanned[-1]

        # Generate the full range of numbers
        total_bk_stats = list(range(start_number, end_number + 1))

        # Find the missing numbers
        bk_stats_missing = sorted(set(total_bk_stats) - set(bk_stats_scanned))

        # Display the results
        print("\n✅ File processed successfully!")
        print(f"📌 Given range: {start_number} to {end_number}")
        print(f"🔢 Total missing numbers: {len(bk_stats_missing)}")
        print(f"📚 Missing book numbers: {bk_stats_missing}")

    except ValueError as ve:
        print(f"\n❌ ValueError: {ve}")
    except FileNotFoundError:
        print(f"\n❌ File not found: {uploaded_filename}")
    except KeyError:
        print(f"\n❌ Error: Unable to read the first sheet from '{uploaded_filename}'. Please check the file.")
    except Exception as e:
        print(f"\n❌ An error occurred while processing '{uploaded_filename}': {e}")

# Create a file upload widget
upload_widget = widgets.FileUpload(
    accept='.xlsx',  # Only accept Excel files
    multiple=False   # Allow only a single file
)

# Define a callback for when the file is uploaded
def on_upload(change):
    if change['new']:
        process_file(change['new'])

# Display message and file upload widget
print("📂 Please upload an Excel file (.xlsx) to check for missing numbers in the first sheet.")
display(upload_widget)
upload_widget.observe(on_upload, names='value')


📂 Please upload an Excel file (.xlsx) to check for missing numbers in the first sheet.


FileUpload(value={}, accept='.xlsx', description='Upload')